# Uniqueness Studies 

In this notebook, we study the exact reconstruction from noiseless measurements, to understand better how many and what kind of measurements we need to reconstruct a trajectory. 

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
from measurements import get_measurements

def find_n_feasible(traj, env, mask, method='least_squares', verbose=False):
    ''' Given a fixed trajectory and and environment, find out if mask leads 
    to a unique solution. 
    
    :param traj: Trajectory instance.
    :param env: Environment instance. 
    :param mask: binary measurements mask.
    :param method: one of the methods accepted by exactSolution.py
    
    :return: 0 for no solution, -1 for multiple solutions, 1 for unique solution. 
    '''
    n_samples = mask.shape[0]
    
    basis, D_complete = get_measurements(trajectory, environment, n_samples=n_samples)
    
    D_missing = D_complete * mask
    
    nonzero_indices = np.unique(np.where(mask > 0)[0])
    
    reduced_basis = basis[:, nonzero_indices]
    D_missing = D_missing[nonzero_indices, :]
    mask = mask[nonzero_indices, :]
    
    if verbose:
        print('find_n_feasible: mask is ', mask)
    
    try:
        X_list = exactSolution(D_missing, environment.anchors, reduced_basis, method, verbose=True)
        err = np.sum(np.abs(X_list[0] - trajectory.coeffs))
        
        if len(X_list) > 1 or  err > 1e-3:  # found multiple solutions
            #print(err, X_list[0], trajectory.coeffs)
            return -1 
        else: # found only one solution
            return 1
        
    except ValueError: # did not find a solution. This has to do with the solver, 
                       # not with the geometric setup. 
        return 0

In [ ]:
def get_n_measurements(mask):
    sum_per_row = np.sum(mask, axis=1)
    
    a = 1.0
    b = 1.0
    c = 0.45
    
    n_measurements = 0
    for r in sum_per_row:
        if r==1:
            n_measurements += a
        elif r==2:
            n_measurements += a+b
        elif r==3:
            n_measurements += a+b+c
    return round(n_measurements, 4)
    

def plot_feasible(mask, ax=None, verbose=False):
    n_feasible = find_n_feasible(trajectory, environment, mask, verbose=verbose)
    
    if ax is None:
        plt.figure(figsize=(5, 3))
        ax = plt.gca()
        
    ax.matshow(mask, vmin=0, vmax=1)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    if n_feasible == 1:
        ax.set_title('feasible')
    else: 
        ax.set_title('not feasible')
        
        
    number = get_n_measurements(mask)
    ax.set_xlabel('#: {}'.format(number))
    return 
        

## Example Setup

A sample setup and reconstruction result.

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_positions = 10 #number of robot sample positions
n_complexity = 3 #model complexity
dim = 2 # dimension of setup. 

trajectory = Trajectory(n_complexity, dim=dim, model='polynomial')
trajectory.set_coeffs(seed=2)
environment = Environment(n_anchors)
environment.set_random_anchors(seed=2)

basis = trajectory.get_basis(n_samples=n_positions)

sample_points = trajectory.get_sampling_points(basis=basis)

D_complete = get_D_topright(environment.anchors, sample_points)
mask = create_mask(n_positions, n_anchors, strategy='single', 
                   dim=dim, n_complexity=n_complexity, seed=2)
D_missing = np.multiply(D_complete, mask)

X = exactSolution(D_missing, environment.anchors, basis, 'least_squares', verbose=True)
traj = trajectory.copy()
traj.set_coeffs(coeffs=X[0])

plt.figure()
environment.plot()
trajectory.plot(basis, color='orange', label='ground truth')
traj.plot(basis, color='blue', marker='x', linestyle=':', label='reconstructed')
plt.legend()

# Solvers

In this section we try a few different solvers. It turns out that least_squares works best, but I keep these tests just in case we want to change at a later point. 


### Solver choice

In [ ]:
# Problem: this actually returns garbage if the first estimate is not feasible. 
# See here: https://github.com/scipy/scipy/issues/7618
#method = 'minimize'  

# Problem: this returns the least squares estimate, but does not impose the quadratic constraints. 
# the residuals of this might be non-zero.
method = 'least_squares'

# Problem: produces memory error even for small problem size. 
#method = 'grid'

# Problem: can only take into account exactly K*dim constraints. 
#method = 'roots'

### Mask strategies

The below strategies are implemented in the measurements module. I noticed however that they might not be the best choice, and I have tested more strategies in this notebook. 

Once we converge on uniqueness results etc. we can move code from this notebook to the measurements module. 

In [ ]:
# the first point sees dim+1 anchors, the next K-2 points see the first two anchors, the Kth point sees only one. 
#strategy = 'simple'

# each point sees exactly one anchor. at least d+1 anchors are seen.
strategy = 'single'

# uniformly delete a few 
#strategy = 'uniform'

In [ ]:
n_it = 100
n_positions = 2 * n_complexity
for i in range(n_it):
    # create random trajectory
    trajectory.set_coeffs(seed=i)
    basis = trajectory.get_basis(n_samples=n_positions)
    sample_points = trajectory.get_sampling_points(basis=basis)

    D_complete = get_D_topright(environment.anchors, sample_points)
    mask = create_mask(n_positions, n_anchors, strategy=strategy, dim=dim, n_complexity=n_complexity, seed=i)
    
    # trying to get more unique solutions by adding 
    # mask[-1, -1] = 1.0
    
    D_missing = np.multiply(D_complete, mask)

    try:
        X = exactSolution(D_missing, environment.anchors, basis, method, verbose=True)
        if method == 'least_squares':
            X = X[0]
        assert np.allclose(X, trajectory.coeffs, atol=1e-3)
        
        #X_noiseless = exactSolution(D_complete, environment.anchors, basis, method)
        #assert np.allclose(X_noiseless, trajectory.coeffs)
        print('Seed {} ok.'.format(i))
        
    except ValueError as e:
        print('ValueError for seed {}:{}'.format(i, e))
        
    except AssertionError as e:
        print('Result not exact for seed:', X, trajectory.coeffs)
        
        # We found an example of inexact reconstruction. 
        # We can thus stop here and continue plotting and 
        # investigating it. 
        break
        
    except Exception as e:
        print('Singular matrix for seed {}? Error message:'.format(i)) 
        raise e

if method == 'roots':
    from exact_solution import objective_root
    print('objective_root found:')
    print(objective_root(X, environment.anchors, basis, D_missing))
    print('objective_root original:')
    print(objective_root(trajectory.coeffs, environment.anchors, basis, D_missing))

### Plot ambiguous result

We stopped above iterations at an ambiguous reconstruction. Now let's plot it to see what it looks like. 

In [ ]:
traj_second_solution = trajectory.copy()
traj_second_solution.set_coeffs(coeffs=X)

plt.figure()
plt.title('plot of two ambiguous solutions')

trajectory.plot(basis, mask=mask, color='green')
trajectory.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
trajectory.plot_number_measurements(basis, mask, legend=True)

traj_second_solution.plot(basis, mask=mask, color='red')
traj_second_solution.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
traj_second_solution.plot_number_measurements(basis, mask, legend=False)
environment.plot()
environment.annotate()

plt.axis('equal')
#plt.xlim([0, 6])
#plt.ylim([0, 6])

# Weighting strategies

The above weighting strategies do not cover all possible cases, so I prototyped the below function. It should be moved to the measurements.py module once we are done analyzing. 

In [ ]:
def get_mask(n_anchors, n_complexity, n_samples, 
             dim, init_method=0, fill_method='', d=1):
    ''' Create a binary mask defining missing measurements. 
    
    :param init_method: different initialization techniques. 
      - 1: we see d+1 anchors from one point (is not enough)
      - 2: we see d+1 anchors from d+1 different points (necessary condition in 2D) 
    if not given, we proceed directly to fill_method. 
      
    :param fill_method: after initializing, choose this method to place the missing samples. 
      - Aa: all missing samples are taken from one of the already seen anchors. 
      - Ab: all missing samples are taken from one anchor not seen yet. 
      - B: requries parameter d. Each point sees d randomly chosen anchors, until we reach required number.
      
    :param d: parameter for fill_method B. 
    
    :return: binary mask of shape n_samples x n_anchors
    '''
    
    assert n_anchors >= dim + 1, 'Need at least dim+1 anchors.'
    
    # get dim+1 different anchor indicies
    anchors_seen = np.random.choice(range(n_anchors), size=(dim+1), replace=False)
    
    # Assumption: 1., 2., 3. do not matter. 
    # 1. all seen from one point
    points_seen = None
    if init_method == 1: 
        idx = 0
        points_seen = [idx] * len(anchors_seen)
    # 2. all seen from d+1 different points
    elif init_method == 2: 
        points_seen = np.random.choice(range(n_samples), size=len(anchors_seen), replace=False)
        
    else:
        points_seen = ()
        anchors_seen = ()
        
    if points_seen is None:
         raise ValueError('no valid mask found.')
        
    mask = np.zeros((n_samples, n_anchors))
    for p, a in zip(points_seen, anchors_seen):
        mask[p, a] = 1.0
    
    anchors_not_seen = list(set(range(n_anchors)) -  set(anchors_seen))
    points_not_seen = np.where(np.sum(mask, axis=1)==0)[0]
    n_missing = n_samples - int(np.sum(mask))
    if n_missing < 0:
        print('Warning: already have {} samples after initialization. More than required {}'.format(np.sum(mask), n_samples))
        return mask
    
    # each position sees the same anchor(s)
    if 'A' in fill_method: 
        
        if len(anchors_not_seen) < 0 and fill_method == 'Ab':
            print('warning: there are no unseen anchors left. Changing method from Ab to Aa')
            fill_method = 'Aa'
        
        # A.a this anchor is part of previously seen
        if fill_method == 'Aa':
            new_anchors = np.random.choice(anchors_seen, size=1, replace=False)
    
        # A.b this anchor is one that was not seen yet
        elif fill_method == 'Ab':
            new_anchors = np.random.choice(anchors_not_seen, size=1, replace=False)
        
        points_left = np.random.choice(points_not_seen, size=n_missing, replace=False) 
        new_anchors = [new_anchors] * len(points_left) 
        mask[points_left, new_anchors] = 1.0
    
    # B. We have in total n_positions measurements, but each point sees 
    # exactly d anchors. 
    elif fill_method == 'B': 
        
        num_full = math.floor(n_missing / d)
        residual = n_missing % d
        print('filling {} with d anchors, and one with {}'.format(num_full, residual))

        num_all = num_full+1 if residual > 0 else num_full

        assert num_full * d + residual == n_missing

        # fill full ones
        new_points = np.random.choice(points_not_seen, size=num_all, replace=False)
        new_points = [[p]*d for p in new_points]
        anchors = [np.random.choice(range(n_anchors), size=d, replace=False) for _ in range(num_full)]
        
        mask[new_points[:num_full], anchors] = 1.0

        # fill residual.
        anchor = np.random.choice(range(n_anchors), size=residual, replace=False)
        mask[new_points[-1][:residual], anchor] = 1.0
        
    else:
        pass
   
    if fill_method != '':
        assert np.sum(mask) == n_samples, '{} != {}'.format(np.sum(mask), n_samples)

    return mask

In [ ]:
dim = trajectory.dim
n_anchors = environment.n_anchors
n_complexity = trajectory.n_complexity
n_samples = 10
fig, axs = plt.subplots(1, 2)
axs[0].matshow(get_mask(n_anchors, n_complexity, n_samples, dim, 1))
axs[0].set_xlabel('init method 1')
axs[1].matshow(get_mask(n_anchors, n_complexity, n_samples, dim, 2))
axs[1].set_xlabel('init method 2')

In [ ]:
# not sure if the following make any difference. 
maskAa = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=2, fill_method='Aa')
maskAb = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=2, fill_method='Ab')
fig, axs = plt.subplots(1, 2)
axs[0].matshow(maskAa)
axs[0].set_xlabel('init method 2, fill method Aa')

axs[1].matshow(maskAb)
axs[1].set_xlabel('init method 2, fill method Ab')

In [ ]:
maskB1 = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=0, fill_method='B', d=1)
maskB2 = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=0, fill_method='B', d=2)
maskB3 = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=0, fill_method='B', d=3)
# show result.
fig, axs = plt.subplots(1, 3)
axs[0].matshow(maskB1); axs[0].set_xlabel('fill method B, d=1')
axs[1].matshow(maskB2); axs[1].set_xlabel('fill method B, d=2')
axs[2].matshow(maskB3); axs[2].set_xlabel('fill method B, d=3')

# Randomized Experiments

In [ ]:
d = 1 # parameter used for fill_method=B only. 

#n_complexity=4; n_anchors=4
n_complexity=3; n_anchors=3
#n_complexity=2; n_anchors=4
dim = 2

trajectory = Trajectory(n_complexity, dim=dim, model='polynomial')
trajectory.set_coeffs(seed=2)
environment = Environment(n_anchors, dim=dim)
environment.set_random_anchors(seed=2)

In [ ]:
# Experiment 1

# for dim=2, worked for n_complexity=2:4
# for dim=3, not working...
init_method = 2  # d+1 different points see d+1 different anchors. 
fill_method = 'Aa' # rest sees one of above anchors.

#n_samples = n_complexity * dim + 1 # zero fail rate
n_samples = n_complexity * dim  # 100 % fail rate

In [ ]:
# Experiment 2
# always rank 4
#init_method = 2  # d+1 different points see d+1 different anchors
#fill_method = 'Ab' # rest sees unseen anchor. 
#n_samples = n_complexity * dim + 1 # zero fail rate
#n_samples = n_complexity * dim  # 70% fail rate

# Experiment 3
# results in rank 2, and 100 % fails. Same for Aa, Ab. 
#init_method = 1  # one points sees d+1 different anchors
#fill_method = 'Aa' # rest sees one unseen anchor. 
#n_samples = n_complexity * (dim + 1) # 100 % fail rate
#n_samples = n_complexity * dim + 1 # 100 % fail rate

# Experiment 4
# sometimes yields rank 2, sometiems 3. 
#init_method = 3  # random anchors, at most duplicate 
#fill_method = 'Aa' # rest sees one unseen anchors. 
#n_samples = n_complexity * dim + 1 # 20 % fail rate, whenever rank 2!
#n_samples = n_complexity * dim # 40 % fail rate, whenever rank 2!
#n_samples = n_complexity * dim - 1 # 85 % fail rate, for rank 3 and rank 2

In [ ]:
# try B method with different numbers...

# Experiment 1
# working for both dim=2 and dim=3.
#init_method = 0 
#fill_method = 'B' # rest sees one unseen anchors. 
#d = 1
#n_samples = n_complexity * dim + 1 # 5% zero fail rate, when rank 2
#n_samples = n_complexity * dim # 100 % fail rate
#n_samples = n_complexity * dim - 1 # 100 % fail rate, not many solutions found. 

# Experiment 2
#init_method = 0
#fill_method = 'B' # rest sees one unseen anchors. 
#d = 2
#n_samples = n_complexity * dim + 1 # 5 % zero fail rate, when rank is 2. 
#n_samples = n_complexity * dim # 100 % fail rate for rank 2 and 3
#n_samples = n_complexity * dim - 1 # 100 % fail rate, not many solutions found. 

# Experiment 3
#init_method = 0
#fill_method = 'B'
#d = 3
#n_samples = n_complexity * (dim+1) + 1 # zero fail rate
#n_samples = n_complexity * (dim+1) # zero fail rate
#n_samples = n_complexity * (dim+1) - 1 # 100% fail rate

In [ ]:
print('n_complexity={}, n_anchors={}, n_samples={}'.format(n_complexity, n_anchors, n_samples))

n_it = 20
n_fails = 0
n_skips = 0
for i in range(n_it):
    mask = get_mask(n_anchors, n_complexity, n_samples, dim, 
                    init_method, fill_method, d=d) 
    
    n_feasible = find_n_feasible(trajectory, environment, mask)
    
    if n_feasible < 0:
        print('{} failed'.format(i))
        n_fails += 1
    elif n_feasible == 0:
        print('{} skipped'.format(i))
        n_skips += 1
    else:
        print('{} ok'.format(i))
    
if n_skips == n_it:
    print('no solutions found.')
else:
    print('fail rate: {} %'.format(n_fails / (n_it - n_skips) * 100))

In [ ]:
# redo the reconstruction for one case where it failed. 
n_samples = mask.shape[0]
basis, D_complete = get_measurements(trajectory, environment, n_samples=n_samples)
D_missing = D_complete * mask
nonzero_indices = np.unique(np.where(mask > 0)[0])
reduced_basis = basis[:, nonzero_indices]
D_missing = D_missing[nonzero_indices, :]
mask = mask[nonzero_indices, :]

plt.matshow(mask)
X_list = exactSolution(D_missing, environment.anchors, reduced_basis, 'least_squares', verbose=True)

plt.figure()
trajectory.plot(reduced_basis, label='ground truth')
for i, X in enumerate(X_list): 
    trajectory_alternative = trajectory.copy()
    trajectory_alternative.set_coeffs(coeffs=X)
    trajectory_alternative.plot(reduced_basis, label='solution {}'.format(i))
    if i == 2:
        break
print(X_list)

# Controlled experiments

## Question1

### In the base case where we have only K sampling points, we need exactly d+1 measurements to different anchors from each. So a total of K(d+1) measurements. In the oppposite case, where each sampling point can only see one anchor, we seem to need dK+1 measurements, so less. Where does this difference come from? It looks like new sampling points impose more constraints than extra measurements at previous points... 

In [ ]:
n_complexity = 3
dim = 2
n_anchors = dim + 1

trajectory = Trajectory(n_complexity=2, dim=dim)
trajectory.set_coeffs(seed=1)

environment = Environment(n_anchors=n_anchors, dim=dim)
environment.set_random_anchors(seed=1)

# Base case: should work
mask = np.zeros((n_complexity, n_anchors))
mask[:n_complexity, :dim+1] = 1.0
n_feasible = find_n_feasible(trajectory, environment, mask)
print('feasible solutions:', n_feasible)

# Study nr. 1: move one from base case.

mask_one = np.r_[mask, np.zeros((1, n_anchors))]
for i, j in zip(*np.where(mask_one == 0)):
    print(i, j)
    new_mask = mask_one.copy()
    new_mask[0, 0] = 0.0
    new_mask[i, j] = 1.0
    
    n_feasible = find_n_feasible(trajectory, environment, new_mask)
    if n_feasible == 0:
        plt.matshow(new_mask)
        plt.title('no solution')
    elif n_feasible < 0:
        plt.matshow(new_mask)
        plt.title('multiple solutions')

In [ ]:
import itertools
# Study nr. 2: move two from base case.

n_choose = 2
#n_choose = 3

mask_two = np.r_[mask, np.zeros((n_choose, n_anchors))]

left_pairs = np.array([*np.where(mask_two == 0)]).T

for pairs in itertools.combinations(left_pairs, n_choose):
    new_mask = mask_two.copy()
    new_mask[0, :n_choose] = 0.0
    
    for pair in pairs:
        new_mask[pair[0], pair[1]] = 1.0
    
    n_feasible = find_n_feasible(trajectory, environment, new_mask)
    if n_feasible == 0:
        plt.matshow(new_mask)
        plt.title('no solution')
    elif n_feasible < 0:
        plt.matshow(new_mask)
        plt.title('multiple solutions')

OK, so it looks like as long as we see d+1 anchors from d+1 different points, we are good. How about the number of meaurements? 

In [ ]:
        
fig, axarr = plt.subplots(2, 6, figsize=(10, 6))
axarr = axarr.reshape((-1, ))
    
i = 0

mask = np.zeros((n_complexity, n_anchors))
mask[:n_complexity, :dim+1] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[0, 0] = 0 
plot_feasible(mask, axarr[i])
i += 1

mask = np.r_[mask, np.zeros((1, n_anchors))]
mask[0, 0] = 0.0
mask[-1, 0] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[0, 1] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask = np.r_[mask, np.zeros((1, n_anchors))]
mask[-1, 1] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[0, 2] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask = np.r_[mask, np.zeros((1, n_anchors))]
mask[-1, 2] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[1, 0] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask[1, 1] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask = np.r_[mask, np.zeros((1, n_anchors))]
mask[-1, 0] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[1, 2] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask = np.r_[mask, np.zeros((1, n_anchors))]
mask[-1, 1] = 1.0
plot_feasible(mask, axarr[i])
i += 1

[ax.axis('off') for ax in axarr[i:]]

In [ ]:
#fig.savefig('plots/feasible_evolution_{}d.png'.format(dim), bbox_inches='tight')

In [ ]:
fig, axarr = plt.subplots(1, 5)
fig.set_size_inches(10, 4)

i = 0
mask[0, -1] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask = np.r_[mask, np.zeros((1, n_anchors))]
mask[-1, 2] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[-1, -1] = 1.0
plot_feasible(mask, axarr[i])
i += 1

mask[-1, 2] = 0.0
plot_feasible(mask, axarr[i])
i += 1

mask[1, -1] = 0.0
plot_feasible(mask, axarr[i])
i += 1

In [ ]:
#fig.savefig('plots/feasible_evolution_{}d_bis.png'.format(dim), bbox_inches='tight')

## Question2

### Can we find the "value" of each constraint type? 2D plane to circle: a, circle to two points: b, two points to point: c. ? 

###  Methodology: I fix K and d, and I generate a few working and failling examples. Each example represents a constraint on a, b, c and the minimum number of measurement "units". Then I solve a linear program to find the values (see linprog.py).  

### Answer: if for example c=0.45, then a=b=1.0 and we need exactly 7.0 or more measurement units for the solution to be defined. 


In [ ]:
fig, axarr = plt.subplots(2, 7)
fig.set_size_inches(9, 6)
axarr = axarr.flatten()
i = 0

n_complexity = 3

dim = 2
n_anchors = dim + 1

trajectory = Trajectory(n_complexity=n_complexity, dim=dim)
trajectory.set_coeffs(seed=1)

environment = Environment(n_anchors=n_anchors, dim=dim)
environment.set_random_anchors(seed=1)

mask = np.ones((n_complexity, n_anchors))
mask = np.r_[mask, np.zeros((3, n_anchors))]
plot_feasible(mask, ax=axarr[i]); i+=1

mask[2, -1] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

# add one to make it feasible
mask[3, 0] = 1
plot_feasible(mask, ax=axarr[i]); i+=1

# create unfeasible
mask[2, 1] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

mask[2, 1] = 1
mask[1, -1] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

# feasible
mask[0, 0] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

# create unfeasible
mask[2, 1] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

# create unfeasible
mask[2, 1] = 1
mask[3, 0] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

# make feasible: 
mask[2, 1] = 0
mask[-3, 0] = 1
mask[-2, 0] = 1
plot_feasible(mask, ax=axarr[i]); i+=1

# make infeasible: 
mask[0, 1] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

mask[-1, 0] = 1
mask[0, 1] = 0
plot_feasible(mask, ax=axarr[i]); i+=1

mask[1, 0] = 0.0
plot_feasible(mask, ax=axarr[i]); i+=1

mask = np.r_[mask, np.zeros((1, 3))]
mask[-1, 0] = 1.0
plot_feasible(mask, ax=axarr[i]); i+=1

[ax.axis('off') for ax in axarr[i:]]